In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pickle  # For saving and loading the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer
import optuna
from sklearn.ensemble import RandomForestClassifier

import os
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import optuna
import joblib
import time
import numpy as np
pd.options.display.float_format = '{:.3f}'.format

/home/ssilvera/Escritorio/Proyectos/Modelo_pagos/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_excel("../Preprocessed data.xlsx")
#df.columns


In [3]:
import os
import joblib
import pandas as pd
import numpy as np
import optuna
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# Definir columnas numéricas a estandarizar
numeric_cols = ["CAPITAL", "CONTEO_PRODUCTOS", "Cupo", "DiasMejorGestion", "DiasUltimaGestion", "EDAD", 'GRUPO', 'Mora_maxima_cliente__Asignacion', 'PgMin', 'SaldoCliente', 'SaldoVencido', 'TotalGestiones', 'TotalGestionesCD', 'TotalGestionesCI', 'TotalGestionesCompromiso', 'TotalGestionesMasivas', 'TotalGestionesNC', 'dias_mora_Asignacion', 'saldo_total']

# Separar características y variable objetivo
X = df.drop(columns=["Cuenta", "Estado"])  # Excluir identificador y objetivo
y = df["Estado"]

# División en train, val y test (estratificado)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Aplicar StandardScaler solo a las columnas numéricas
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Guardar el scaler
scaler_filename = "../Models/scaler_2.pkl"
joblib.dump(scaler, scaler_filename)

# Definir nombres de archivos
model_filename = "../Models/mlp_model_2.keras"
study_filename = "../Study/optuna_study_MLP_2.pkl"

# Obtener el número de características después del preprocesamiento
input_dim = X_train.shape[1]

# Función objetivo para Optuna
def objective(trial):
    # Espacio de hiperparámetros
    n_layers = trial.suggest_int("n_layers", 1, 3)
    units = trial.suggest_int("units", 32, 256)
    dropout_rate = trial.suggest_float("dropout_rate", 0.05, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
    
    # Construcción del modelo
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    
    for _ in range(n_layers):
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(len(y.unique()), activation="softmax"))  # Salida con clases
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Entrenamiento con Early Stopping
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        verbose=0,
        callbacks=[early_stopping]
    )

    # Evaluación en validación
    y_val_pred = np.argmax(model.predict(X_val), axis=1)
    f1 = f1_score(y_val, y_val_pred, average="weighted")

    return f1

# Intentar cargar el estudio de Optuna si existe
if os.path.exists(study_filename):
    print(f"Study found. Loading from {study_filename}...")
    study_mlp = joblib.load(study_filename)
else:
    print("Study not found. Creating a new one...")
    pruner = optuna.pruners.MedianPruner()
    study_mlp = optuna.create_study(direction="maximize", pruner=pruner)

    # Ejecutar la optimización
    start_time = time.time()
    study_mlp.optimize(objective, n_trials=50)
    end_time = time.time()

    # Guardar el estudio de Optuna
    joblib.dump(study_mlp, study_filename)
    print(f"Study saved as {study_filename}")

    print("Best trial:")
    trial = study_mlp.best_trial
    print("  Value:", trial.value)
    print("  Params:", trial.params)
    print(f"Optimization time: {end_time - start_time:.2f} seconds")

# Obtener los mejores parámetros de la optimización
best_params = study_mlp.best_params

# Entrenar el modelo final con los mejores parámetros
final_model = keras.Sequential()
final_model.add(layers.Input(shape=(input_dim,)))

for _ in range(best_params["n_layers"]):
    final_model.add(layers.Dense(best_params["units"], activation="relu"))
    final_model.add(layers.Dropout(best_params["dropout_rate"]))

final_model.add(layers.Dense(len(y.unique()), activation="softmax"))

final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_params["learning_rate"]),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Guardar el modelo entrenado
final_model.save(model_filename)
print(f"Model saved as {model_filename}")

# Evaluación en validación
y_val_pred = np.argmax(final_model.predict(X_val), axis=1)

print("\nValidation Metrics:")
print(f"F1-Score (Validation): {f1_score(y_val, y_val_pred, average='weighted'):.4f}")
print(classification_report(y_val, y_val_pred))

# Matriz de confusión en validación
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
disp_val = ConfusionMatrixDisplay(conf_matrix_val, display_labels=final_model.predict_classes(X_val))
disp_val.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix (Validation Set)")
plt.show()


2025-02-08 13:15:09.296975: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-08 13:15:09.307461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739038509.322330  740841 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739038509.324907  740841 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 13:15:09.333976: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Study not found. Creating a new one...


/tmp/ipykernel_740841/1076475844.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
W0000 00:00:1739038510.454449  740841 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step


[I 2025-02-08 13:16:07,395] Trial 0 finished with value: 0.8338268088898635 and parameters: {'n_layers': 2, 'units': 87, 'dropout_rate': 0.054033111531859544, 'learning_rate': 1.2701170001545189e-05}. Best is trial 0 with value: 0.8338268088898635.
/tmp/ipykernel_740841/1076475844.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)


...

[I 2025-02-08 13:41:12,891] Trial 48 finished with value: 0.8563993021952259 and parameters: {'n_layers': 2, 'units': 170, 'dropout_rate': 0.4897163669048524, 'learning_rate': 0.00016885891366260215}. Best is trial 7 with value: 0.8585784441999732.
/tmp/ipykernel_740841/1076475844.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)


149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step


[I 2025-02-08 13:42:03,355] Trial 49 finished with value: 0.857356247237514 and parameters: {'n_layers': 3, 'units': 124, 'dropout_rate': 0.167015587577238, 'learning_rate': 6.492147332863847e-05}. Best is trial 7 with value: 0.8585784441999732.


Study saved as ../Study/optuna_study_MLP_2.pkl
Best trial:
  Value: 0.8585784441999732
  Params: {'n_layers': 3, 'units': 237, 'dropout_rate': 0.46335009555914025, 'learning_rate': 8.697489751293547e-05}
Optimization time: 1612.95 seconds
Epoch 1/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5813 - loss: 1.1591 - val_accuracy: 0.7573 - val_loss: 0.6522
Epoch 2/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6974 - loss: 0.7535 - val_accuracy: 0.7720 - val_loss: 0.5913
Epoch 3/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7293 - loss: 0.6610 - val_accuracy: 0.7850 - val_loss: 0.5733
Epoch 4/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7485 - loss: 0.6180 - val_accuracy: 0.7921 - val_loss: 0.5524
Epoch 5/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7670 - loss: 0.5794 - val_accuracy: 0.7948 - val_loss: 0.5332
Epoch 6/100
696/696 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7696 - loss: 0.5698 - val_accuracy: 0.7984 - val_

AttributeError: 'Sequential' object has no attribute 'predict_classes'